## Berechnungen

In diesem Notebook ist es möglich den Mittelwert, Varianz und Standardabeichung für Messwerte zu berechnen.   
Zudem wird die Gieränderung in rad bestimmt. Die Ergebnisse können zurück in die Merkmal CSV Datei geschrieben werden.

In [24]:
import pandas
import numpy as np
import numpy.testing as npt
import math
import os

In [44]:
featuresDf = pandas.read_csv("merkmaleRoh.csv")
featuresDf.describe()

,Zeitstempel,Breitengrad,Laengengrad,Geschwindigkeit,Messwerte,StartZeitfenster,StartBewegungsdaten,StartBelichtung,Belichtungszeit
count,9.919000e+03,9919.000000,9919.000000,9919.000000,9919.000000,9.919000e+03,9.919000e+03,9.919000e+03,9.919000e+03
mean,1.521032e+12,53.629287,10.149711,13.872899,18.853513,6.647388e+13,6.647381e+13,6.647391e+13,1.507954e+07
std,8.739765e+05,0.010715,0.007430,3.287708,0.792188,8.739765e+11,8.739767e+11,8.739762e+11,6.355233e+06
min,1.521030e+12,53.613064,10.137839,5.040000,14.000000,6.486842e+13,6.486834e+13,6.486845e+13,5.644420e+06
25%,1.521031e+12,53.618630,10.142929,11.880000,18.000000,6.574524e+13,6.574518e+13,6.574527e+13,1.000184e+07
50%,1.521032e+12,53.630350,10.149292,14.184001,19.000000,6.649343e+13,6.649336e+13,6.649346e+13,1.000184e+07
75%,1.521032e+12,53.639490,10.156333,16.200000,19.000000,6.721270e+13,6.721265e+13,6.721273e+13,2.000368e+07
max,1.521033e+12,53.646214,10.163120,24.264000,32.000000,6.795114e+13,6.795109e+13,6.795116e+13,2.999632e+07


In [45]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9919 entries, 0 to 9918
Data columns (total 16 columns):
Zeitstempel            9919 non-null int64
Breitengrad            9919 non-null float64
Laengengrad            9919 non-null float64
Geschwindigkeit        9919 non-null float64
AccelerometerX         9919 non-null object
AccelerometerY         9919 non-null object
AccelerometerZ         9919 non-null object
Azimuth                9919 non-null object
Nick                   9919 non-null object
Roll                   9919 non-null object
SensorZeitstempel      9919 non-null object
Messwerte              9919 non-null int64
StartZeitfenster       9919 non-null int64
StartBewegungsdaten    9919 non-null int64
StartBelichtung        9919 non-null int64
Belichtungszeit        9919 non-null int64
dtypes: float64(3), int64(6), object(7)
memory usage: 1.2+ MB


In [46]:
# Zeigt die ersten 5 Reihen Beschleunigungssensordaten der X-Achse 
featuresDf.AccelerometerX.head()

0    -0.25269 -0.15618 -0.14026 -0.09689 -0.04687 -...
1    0.32043 -0.4485 -0.46607 -0.32688 0.07559 -0.0...
2    1.01087 -0.64698 -0.94662 0.28466 0.15112 0.16...
3    -1.38306 -0.78467 -0.6737 -0.33976 0.29513 0.1...
4    -0.54456 -0.57355 -0.65804 -0.60305 -0.60502 -...
Name: AccelerometerX, dtype: object

Die folgenden Spalten des Datenframes haben als Type ein Python object.   
AccelerometerX       non-null object   
AccelerometerY       non-null object   
AccelerometerZ       non-null object   
Azimuth              non-null object   
Nick                 non-null object   
Roll                 non-null object   
Implizit haben diese Spalten als Typ Strings. Aber in einem DataFrame werden Strings als    
Python objects erkannt. Um den Datentyp auf float zu casten wird aus den Spalten zunächst ein Liste erstellt.    
Welche dann wiederum als Numpy Array auf den Typ float konvertiert wird.   

In [47]:
accXList = featuresDf.AccelerometerX.str.split(" ").tolist()

In [54]:
# Direkt mit dem Numpy Array ist es nicht möglich die Konvertierung durchzuführen, weil die Arrays von Typ object sind.   
pandas.DataFrame(featuresDf.AccelerometerX.str.split(" ")).values[1]

array([ ['0.32043', '-0.4485', '-0.46607', '-0.32688', '0.07559', '-0.06211', '-0.2642', '-1.00815', '-0.72991', '-0.20086', '-0.49778', '-0.76599', '-0.1531', '-0.16845', '-0.41056', '0.02398', '0.38693', '-0.33402', '-0.49706']], dtype=object)

In [51]:
accXList[1] # zeige Liste

[['-0.25269',
  '-0.15618',
  '-0.14026',
  '-0.09689',
  '-0.04687',
  '-0.0375',
  '0.04663',
  '-0.37642',
  '-0.08662',
  '-0.05396',
  '-0.15043',
  '-0.4881',
  '-0.08402',
  '-0.1898',
  '-0.33571',
  '-0.28389',
  '-0.30373',
  '-0.4575',
  '-0.07486'],
 ['0.32043',
  '-0.4485',
  '-0.46607',
  '-0.32688',
  '0.07559',
  '-0.06211',
  '-0.2642',
  '-1.00815',
  '-0.72991',
  '-0.20086',
  '-0.49778',
  '-0.76599',
  '-0.1531',
  '-0.16845',
  '-0.41056',
  '0.02398',
  '0.38693',
  '-0.33402',
  '-0.49706'],
 ['1.01087',
  '-0.64698',
  '-0.94662',
  '0.28466',
  '0.15112',
  '0.16687',
  '0.62383',
  '-0.03724',
  '-0.64271',
  '-0.29965',
  '0.02078',
  '-0.535',
  '-0.30542',
  '-0.27499',
  '-0.43451',
  '-0.42422',
  '-0.04824',
  '-0.46763'],
 ['-1.38306',
  '-0.78467',
  '-0.6737',
  '-0.33976',
  '0.29513',
  '0.12884',
  '-0.00417',
  '0.36441',
  '-0.04558',
  '0.4079',
  '0.43358',
  '0.40816',
  '0.20394',
  '0.66881',
  '0.45843',
  '0.32078',
  '0.74697',
  '0.398

In [52]:
np.array(accXList).dtype

dtype('O')

In [56]:
np.array(accXList).astype(float).dtype 

ValueError: setting an array element with a sequence.

In [57]:
accXNp = np.array(accXList).astype(float) 
accXDf = pandas.DataFrame(accXNp)
accXDf.describe()

ValueError: setting an array element with a sequence.

In [58]:
accXDf.shape

(11277, 20)

In [91]:
# Die Anzahl der Messerte muss immer gleich sein, ansonsten werden fehlende Messwerte
# mit NaN gefüllt.
featuresDf.Messwerte[featuresDf.Messwerte != 20]

Series([], Name: Messwerte, dtype: int64)

In [87]:
# 
unwantedRows = featuresDf.Messwerte[featuresDf.Messwerte != 20]
featuresDf.drop(featuresDf.index, axis=0, inplace=True)

In [89]:
# Konvertierung von Object zu Float Werten in einer Zeile 
accYDf = pandas.DataFrame(np.array(featuresDf.AccelerometerY.str.split(" ").tolist()).astype(float))

In [92]:
accYDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 1 columns):
0    0 non-null float64
dtypes: float64(1)
memory usage: 76.0 bytes


In [101]:
accYDf

,0


In [94]:
accZDf = pandas.DataFrame(np.array(featuresDf.AccelerometerZ.str.split(" ").tolist()).astype(float))

In [95]:
nickDf = pandas.DataFrame(np.array(featuresDf.Nick.str.split(" ").tolist()).astype(float))

In [96]:
rollDf = pandas.DataFrame(np.array(featuresDf.Roll.str.split(" ").tolist()).astype(float))

In [97]:
azimuthDf = pandas.DataFrame(np.array(featuresDf.Azimuth.str.split(" ").tolist()).astype(float))

In [98]:
accXMean = accXDf.T.mean()
accXMean.head(2)

0    0.182809
1    0.079515
dtype: float64

In [21]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
featuresDf['MittelX'] = accXMean
featuresDf['MittelY'] = accYDf.T.mean()
featuresDf['MittelZ'] = accZDf.T.mean()
featuresDf['MittelNick'] = nickDf.T.mean()
featuresDf['MittelRoll'] = rollDf.T.mean()

In [22]:
featuresDf.MittelX.head(2)

0    0.150673
1   -0.007415
Name: MittelX, dtype: float64

In [23]:
featuresDf.columns

Index(['Zeitstempel', 'Breitengrad', 'Laengengrad', 'Geschwindigkeit',
       'AccelerometerX', 'AccelerometerY', 'AccelerometerZ', 'Azimuth', 'Nick',
       'Roll', 'SensorZeitstempel', 'Messwerte', 'StartBewegungsD',
       'StartBelichtung', 'Belichtungszeit', 'MittelX', 'MittelY', 'MittelZ',
       'MittelNick', 'MittelRoll'],
      dtype='object')

Berechnet die Varianz. Dieser Funktion muss als Paramter der Mittelwert (mean) und der DataFrame mit   
den Float Werten übergeben werden, um die Varianz zu berechnen. Als Varianz wird der Durchschnitt der quadrierten  
Differenzen zum Mittelwert bezeichnet.   
Prec.:    
Postc.: Gibt die berechnete Variance als Float zurück oder 0 wenn die Anzahl der Werte <= 0  

In [24]:
def calcVariance(meansDf, dfValues):
    variance = []
    for i, it in dfValues.iterrows():
        sum = 0
        for value in it:
            tempDifference = value-meansDf[i]
            sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / it.count(),5)))
    return np.array(variance).astype(float)

Hier wird die Standardabweichung der Varianz berechnet. Dies ist die Wurzel der Varianz.   
Für die Berechnung wird der Absolutwert von der Varianz genommen.    
Prec.:
Postc.:  Standardabweichung wird zurückgegeben.

In [25]:
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        temp = math.sqrt(np.abs(v))
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

Die Funktion berechnet die Winkeldifferenz in rad zwischen den ersten gemessenen Gierwinkel und den letzten Gierwinkel   
innerhalb einer als Argument übergebenen DataFrames. Der zurückgegebene Radiant    
ist immer positiv und gibt die relative änderung in rad an.      
Prec.:   
Postc.: Radiant berechnet   

In [26]:
def calculateAngelChangeAzimuth(azimuthDf):
    result = []
    #counter = 0
    for i,values in azimuthDf.iterrows():
        #for rad in values:
            #azimuthL.append(rad * (180 / math.pi))
            #counter = counter + 1
            #print(counter)
        first = values[0]*(180/math.pi)
        last = values.iloc[-1]*(180/math.pi)
        resultTemp = np.abs(first - last)
        if(resultTemp > 180):
            resultTemp = 360 - resultTemp
        result.append("{0:.5f}".format(round(resultTemp * ( math.pi / 180),5)))
        #counter = 0
    return np.array(result).astype(float)

In [27]:
featuresDf['AzimuthAenderung'] = calculateAngelChangeAzimuth(azimuthDf)

In [28]:
# Berechne Varianz
featuresDf['VarianzX'] = calcVariance(featuresDf.MittelX,accXDf)
featuresDf['VarianzY'] = calcVariance(featuresDf.MittelY,accYDf)
featuresDf['VarianzZ'] = calcVariance(featuresDf.MittelZ,accZDf)
featuresDf['VarianzNick'] = calcVariance(featuresDf.MittelNick,nickDf)
featuresDf['VarianzRoll'] = calcVariance(featuresDf.MittelRoll,rollDf)

In [29]:
featuresDf.VarianzX.head()

0    0.40178
1    0.24189
2    0.04570
3    0.08168
4    0.12548
Name: VarianzX, dtype: float64

In [30]:
# Berechne Standardabweichung
featuresDf['AbweichungX'] = calculateStandardDeviation(featuresDf.VarianzX)
featuresDf['AbweichungY'] = calculateStandardDeviation(featuresDf.VarianzY)
featuresDf['AbweichungZ'] = calculateStandardDeviation(featuresDf.VarianzZ)
featuresDf['AbweichungNick'] = calculateStandardDeviation(featuresDf.VarianzNick)
featuresDf['AbweichungRoll'] = calculateStandardDeviation(featuresDf.VarianzRoll)

In [31]:
featuresDf.AbweichungX.head()

0    0.63386
1    0.49182
2    0.21378
3    0.28580
4    0.35423
Name: AbweichungX, dtype: float64

In [32]:
accXDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12149 entries, 0 to 12148
Data columns (total 20 columns):
0     12149 non-null float64
1     12149 non-null float64
2     12149 non-null float64
3     12149 non-null float64
4     12149 non-null float64
5     12149 non-null float64
6     12149 non-null float64
7     12149 non-null float64
8     12149 non-null float64
9     12149 non-null float64
10    12149 non-null float64
11    12149 non-null float64
12    12149 non-null float64
13    12149 non-null float64
14    12149 non-null float64
15    12149 non-null float64
16    12149 non-null float64
17    12149 non-null float64
18    12149 non-null float64
19    12149 non-null float64
dtypes: float64(20)
memory usage: 1.9 MB


In [33]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.21672,0.42177,-0.35210,-0.86394,-1.08952,0.06309,0.84727,0.92300,0.95294,0.90027,0.22990,-0.33705,-0.33092,-0.46391,-0.55499,-0.09155,0.21790,0.28160,0.56240,0.48058
1,0.54781,0.31568,0.06868,-0.03698,-0.38200,-0.93382,-0.62446,0.02143,0.81395,1.06489,0.57611,0.20042,-0.13079,-0.42639,-0.69353,-0.03384,-0.39480,-0.11662,0.05994,-0.04397
2,-0.19796,-0.23496,-0.24925,-0.29133,-0.27901,-0.14658,-0.05596,-0.01411,-0.01127,0.05229,0.37895,0.24189,0.17820,0.15790,0.01907,-0.26054,-0.03987,0.07538,-0.35340,0.36086
3,0.14269,0.03755,0.13732,0.33972,0.40969,0.06728,-0.22198,-0.26951,-0.35350,0.25352,0.11089,-0.40161,-0.09143,-0.33361,-0.03703,0.33814,-0.58755,0.41869,0.25836,-0.00783
4,0.30531,0.52007,-0.02829,-0.00730,0.33128,0.08117,0.17221,0.13778,-0.65591,-0.26423,-0.15008,-0.48779,-0.09908,-0.64620,-0.22580,-0.48709,-0.46628,-0.77140,-0.47920,-0.26076


In [34]:
featuresDf.MittelX.head()

0    0.150673
1   -0.007415
2   -0.033485
3    0.010490
4   -0.174079
Name: MittelX, dtype: float64

In [35]:
# Änderungen in CSV Datei schreiben
featuresDf.set_index('Zeitstempel', inplace=True)
featuresDf.to_csv('../merkmale.csv')

In [39]:
pandas.read_csv("../merkmale.csv").columns

Index(['Zeitstempel', 'Breitengrad', 'Laengengrad', 'Geschwindigkeit',
       'AccelerometerX', 'AccelerometerY', 'AccelerometerZ', 'Azimuth', 'Nick',
       'Roll', 'SensorZeitstempel', 'Messwerte', 'StartBewegungsD',
       'StartBelichtung', 'Belichtungszeit', 'MittelX', 'MittelY', 'MittelZ',
       'MittelNick', 'MittelRoll', 'AzimuthAenderung', 'VarianzX', 'VarianzY',
       'VarianzZ', 'VarianzNick', 'VarianzRoll', 'AbweichungX', 'AbweichungY',
       'AbweichungZ', 'AbweichungNick', 'AbweichungRoll'],
      dtype='object')

In [38]:
# Unittests
import unittest

class CalcTest(unittest.TestCase):
    
    
    # Testet die Funktion calcVariance mit einem DataFrame mit den Werten [2.0, 2.0]
    # Der Mittelwert sollte 2.0 sein
    # Das erwartete Ergebnis ist 0.0
    def testCalcVariance2(self):
        meanL = [2.0]
        meanDf = pandas.DataFrame(meanL)
        valuesA = [2.0,2.0]
        valuesDf = pandas.DataFrame([[2.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 0.0,2)
        
    # Testet die Funktion calcVariance mit einem DataFrame mit den folgenden Werten:
    # [1.24, 2.5213, 10.434, 42.45, 5.9]
    # Der Mittelwert ist 12.508
    # Die erwartete Varianz ist 234,247016
    def testCalcVariance5(self):
        meanDf = pandas.DataFrame([12.508])
        valuesDf = pandas.DataFrame([[1.24, 2.52, 10.43, 42.45, 5.9]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 234.24702,5)   
    
    # Testet calcVariance mit 0 Datenwerten   
    def testCalcVariance0(self):
        meanDf = pandas.DataFrame([0.0])
        valuesDf = pandas.DataFrame([[0.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),0.0,2)  
        
    # Testet calcVariance mit negativen Datenwerten   
    def testCalcVarianceNegative(self):
        meanDf = pandas.DataFrame([-24.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),451.5,1) 
        
    # Testet calcVariance mit Muultidimensionalen DataFrame   
    def testCalcVarianceMultidim(self):
        meanDf = pandas.DataFrame([-24.0,0.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0],[0.0,0.0,0.0,0.0]])
        # Hier wird Numpy Test verwendet um das Ergebnis (zwei Arrays) zu vergleichen
        npt.assert_array_equal(calcVariance(meanDf[0],valuesDf),np.array([ 451.5,0.0])) 
        
    # Dieser Test testet die Funktion calculateStandardDeviation.
    def testCalculateStandardDeviation(self):
        varianceDf = pandas.DataFrame([451.5])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),21.24853,5) 
        
    # Testet die Funktion calculateStandardDeviation.
    # Dabei hat das Argument den Wert 0.0.
    def testCalculateStandardDeviationZero(self):
        varianceDf = pandas.DataFrame([0.0])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),0.0,2)
    
    #  Testet die Funktion calculateAngelChangeAzimuth mit zwei gleichen rad Werten
    def testCalculateAngelChangeAzimuthEqual(self):
        radiants = pandas.DataFrame([[1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),0.0,2)
    
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten -1.0 und 1.0
    def testCalculateAngelChangeAzimuthOne(self):
        radiants = pandas.DataFrame([[-1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 1.0 und -1.0
    def testCalculateAngelChangeAzimuthOne2(self):
        radiants = pandas.DataFrame([[1.0,-1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  

    # Testet die Funktion calculateAngelChangeAzimuthValues mit mehreren Werten
    def testCalculateAngelChangeAzimuthValues(self):
        radiants = pandas.DataFrame([[-1.57,1.0,2.0,1.3,1.57]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),3.14,2)
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 0.05 und 0.06
    def testCalculateAngelChangeAzimuthSmall(self):
        radiants = pandas.DataFrame([[0.01,0.02]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 0.01,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde mit 
    # negative Radianten 
    def testCalculateAngelChangeAzimuthNegativ(self):
        radiants = pandas.DataFrame([[-2.9,-2.0,-1.28]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 1.62,2) 

unittest.main(argv=[''], verbosity=2, exit=False)

testCalcVariance0 (__main__.CalcTest) ... ok
testCalcVariance2 (__main__.CalcTest) ... ok
testCalcVariance5 (__main__.CalcTest) ... ok
testCalcVarianceMultidim (__main__.CalcTest) ... ok
testCalcVarianceNegative (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthEqual (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthNegativ (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne2 (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthSmall (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthValues (__main__.CalcTest) ... ok
testCalculateStandardDeviation (__main__.CalcTest) ... ok
testCalculateStandardDeviationZero (__main__.CalcTest) ... ok

----------------------------------------------------------------------
Ran 13 tests in 0.090s

OK
